In [1]:
# placeholder jupyter notebook for the final project

In [2]:
import os


import pandas as pd
import numpy as np

# Assign spreadsheet filename to 'file'
Sales_file = 'Louisville Sales.xlsx'

# Load spreadsheet
salesxl = pd.ExcelFile(Sales_file)

# Print the sheet names
print(salesxl.sheet_names)

['sheet1']


In [3]:
# Load a sheet into a DataFrame by name: df1
df1 = salesxl.parse('sheet1')

Rent_file = 'Rental data.xlsx'
rentxl = pd.ExcelFile(Rent_file)
print(rentxl.sheet_names)

['sheet1']


In [4]:
df2 = rentxl.parse('sheet1')

# writer = pd.ExcelWriter('practice.xlsx', engine='xlsxwriter')
# df2.to_excel(writer, 'Sheet1')
# writer.save()

In [5]:
df1.head()

,List Number,Book Section,Property Sub Type,Listing Date,Sold Date,Contract Acceptance Date,Original List Price,Listing Price,Sold Price,Area,...,End Date,HOA Y/N,Stories,# Upper Floor Bedrooms,First Floor Laundry,Monthly Maintenance,Seller Concessions,HOA Annual $,Closing Costs and Prepaids,Days on Market
0,1573919,Single Family Residence,SF,2020-11-12,2020-12-30,2020-12-18,189950.0,189950.0,189900.0,5,...,2021-05-12,Y,1,0,N,NaN,Y,400.0,4600.0,36
1,1572930,Single Family Residence,SF,2020-10-30,2020-12-31,2020-11-13,184900.0,182900.0,175000.0,5,...,2021-10-30,N,1,0,Y,NaN,N,0.0,NaN,14
2,1574676,Single Family Residence,SF,2020-11-02,2020-11-20,2020-11-02,290000.0,290000.0,290000.0,9,...,2020-11-30,N,1,0,N,NaN,N,0.0,NaN,0
3,1574083,Condominium,CP,2020-11-16,2021-01-08,2020-12-21,69900.0,67000.0,62000.0,3,...,2021-02-15,Y,1,0,N,NaN,N,3153.0,NaN,35
4,1572788,Condominium,CP,2020-10-29,2020-11-30,2020-10-30,115000.0,115000.0,115000.0,7,...,2021-04-29,Y,1,0,Y,NaN,N,2328.0,NaN,1


In [6]:
# calculating subdivision level average price per sqft and average age and average size for both rental and sales

groupped_sales_df = df1.groupby('Subdivision/Condo')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "Sold Price/SqFt": "Average_Sold_Price", 
    "Age": "Average_Sold_Age", 
    "SqFt -Total Finished": "Average_Sold_SqFt"
})

sales_count = df1.groupby('Subdivision/Condo')['Sold Price/SqFt', 'Age', 'SqFt -Total Finished'].size().reset_index(name='sales_count')
#groupped_sales_df1 = pd.merge(groupped_sales_df, sales_count), on="Subdivision/Condo")
groupped_sales_df.head()

,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt
Subdivision/Condo,,,
1400 Willow,335.096667,40.000000,2584.666667
2000 SPRING CONDOMIN,51.030000,64.000000,1113.000000
550 LOFTS,189.930000,97.000000,1211.000000
ABBEY WOODS,131.650000,24.500000,1871.500000
ABERDEEN,208.615714,79.214286,2469.500000


In [7]:
groupped_rental_df = df2.groupby('Subdivision/Condo')['List Price/SqFt', 'Age', 'SqFt -Total Finished'].mean().rename(columns={
    "List Price/SqFt": "Average_Rent_Price", 
    "Age": "Average_Rent_Age", 
    "SqFt -Total Finished": "Average_Rent_SqFt"
})
groupped_rental_df.head()

,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt
Subdivision/Condo,,,
2000 SPRING CONDOMIN,1.030,102.0,1600.0
ABERDEEN,1.155,74.0,1873.0
ACADEMY RIDGE,0.894,15.6,2535.2
ACADEMY RIDGE AT LANDIS LAKE,1.000,17.0,2030.0
ACADEMY RIDGE ESTATES,0.830,18.0,1900.0


In [8]:
subdivision_df = pd.merge(groupped_sales_df, groupped_rental_df, on="Subdivision/Condo")
subdivision_df.head()

,Average_Sold_Price,Average_Sold_Age,Average_Sold_SqFt,Average_Rent_Price,Average_Rent_Age,Average_Rent_SqFt
Subdivision/Condo,,,,,,
2000 SPRING CONDOMIN,51.030000,64.000000,1113.000000,1.030,102.0,1600.000000
ABERDEEN,208.615714,79.214286,2469.500000,1.155,74.0,1873.000000
ACADEMY RIDGE,144.336667,16.666667,2128.333333,0.894,15.6,2535.200000
ACADEMY RIDGE AT LANDIS LAKE,145.187500,17.250000,2661.500000,1.000,17.0,2030.000000
ADAMS RUN,117.815000,18.045455,2193.500000,0.890,18.0,1998.666667
